In [1]:
from src.common.vector import *
from src.common.reference_frame import *
from src.common.time import *
from datetime import datetime

In [1]:
from src.common.vector import *
from src.common.reference_frame import *

r = Vector(-5368,-1784,3691)
(ra,dec) = radec_from_r(r)

print(f'RA = {np.rad2deg(ra)}\nDEC = {np.rad2deg(dec)}')

RA = 198.38370037548617
DEC = 33.12454287112769


In [2]:


date_1 = datetime(2004,5,12,14,45,30)
jd_1 = julian_day(date_1)
date_2 = datetime(1957,10,4,19,26,24)
jd_2 = julian_day(date_2)
print(f'JD_1 = {jd_1} dias\nJD_2 = {jd_2} dias')
print(f'Diferença = {jd_1 - jd_2} dias')


JD_1 = 2453138.1149305557 dias
JD_2 = 2436116.31 dias
Diferença = 17021.804930555634 dias


In [3]:
from src.common.vector import *
from src.common.time import *
from datetime import datetime

theta = sidereal_time(date=datetime(2004,3,3,4,30,0),longitude=139.8)
theta

8.593542535242534

In [2]:
#Exemplo 5.7 - Curtis

from src.common.vector import *
from src.common.reference_frame import *

#Cálculos
r = Vector(x=-5368,y=-1784,z=3691)
alt = 0
lat = 20
lon = 60
r_o = get_topocentric_origin(longitude = 60,latitude = 20,altitude = 0,theta_g=126.7, is_geocentric_latitude=False)
rho = r-r_o
ra,dec = radec_from_r(rho)

#Resultados
print(f'r = {round(r_o.x,0)}I + ({round(r_o.y,1)})J + ({round(r_o.z,0)})K')
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = -5955.0I + (-699.5)J + (2168.0)K
RA = 298.4°
DEC = 51.01°


In [7]:
# Observatório da Aldoria
lat = -30.5262693
lon = -70.8531799
alt = 2180e-3
date = datetime(2023,4,25,9,50,32)
r_origin = get_topocentric_origin(latitude=lat,longitude=lon,altitude=alt,date=date)

ra_object = np.deg2rad(204.6559)
dec_object = np.deg2rad(-45.703182)

r_obj = Vector(x = 1e10*np.cos(dec_object)*np.cos(ra_object),y=1e10*np.cos(dec_object)*np.sin(ra_object),z = 1e10*np.sin(dec_object))

rho = r_origin - r_obj

ra,dec = radec_from_r(rho)


print(f'r = {round(r_origin.x,0)}I + ({round(r_origin.y,1)})J + ({round(r_origin.z,0)})K')
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = 1879.0I + (-5179.8)J + (-3206.0)K
RA = 24.7°
DEC = 45.7°


In [8]:
# Exemplo 5.11 - Curtis

from src.common.vector import *
from src.common.reference_frame import *
from src.common.constants import EARTH_GRAVITATIONAL_PARAMETER

import numpy.polynomial.polynomial as poly

#Parâmetros de entrada:
lat = 40
alt = 1
local_sidereal_times = [44.506,45,45.499]
RAs = [43.537,54.420,64.318]
DECs = [-8.7833,-12.074,-15.105]
times = [0,118.10,237.58]

#Origens dos referências topocêntricos
R1 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[0],is_geocentric_latitude=False)
R2 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[1],is_geocentric_latitude=False)
R3 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[2],is_geocentric_latitude=False)

#Posições relativas (rhos)
versor_rho1 = versor_from_radec(ra = RAs[0], dec = DECs[0])
versor_rho2 = versor_from_radec(ra = RAs[1], dec = DECs[1])
versor_rho3 = versor_from_radec(ra = RAs[2], dec = DECs[2])

#Intervalos de tempo
tau_1 = times[0]-times[1]
tau_3 = times[2]-times[1]
tau = times[2]-times[0]

#Cálculo dos produtos vetoriais:
p1 = cross_product(versor_rho2,versor_rho3)
p2 = cross_product(versor_rho1,versor_rho3)
p3 = cross_product(versor_rho1,versor_rho2)

#Constante D0:
D0 = dot_product(versor_rho1,p1)

#Constantes Ds:
D11 = dot_product(R1,p1)
D12 = dot_product(R1,p2)
D13 = dot_product(R1,p3)
D21 = dot_product(R2,p1)
D22 = dot_product(R2,p2)
D23 = dot_product(R2,p3)
D31 = dot_product(R3,p1)
D32 = dot_product(R3,p2)
D33 = dot_product(R3,p3)

#Constantes A e B:
A = (-D12*(tau_3/tau) + D22 + D32*(tau_1/tau))/D0
B = (1/(6*D0))*(D12*(tau_3**2 - tau**2)*tau_3/tau + D32*(tau**2-tau_1**2)*tau_1/tau)

#Constante E:
E = dot_product(R2,versor_rho2)

#Coeficientes do polnômio (r2)^8 + a(r2)^6 + b(r2)^3 + c:
a = -(A**2 + 2*A*E + R2.magnitude**2)
b = -2*EARTH_GRAVITATIONAL_PARAMETER*B*(A+E)
c = -(EARTH_GRAVITATIONAL_PARAMETER*B)**2

#Raízes do polinomio:
coef = [c,0,0,b,0,0,a,0,1]
roots = poly.polyroots(coef)

#Identificação de r2 dentre as raízes:
condition = (np.isreal(roots)) & (roots > 0)
roots_filtered = roots[condition]
r2_star = np.real(roots_filtered[0])

#Cálculo dos rho's:
rho1_mod = ((6*(D31*tau_1/tau_3 + D21*tau/tau_3)*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*D31*(tau**2-tau_1**2)*tau_1/tau_3)/(6*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*(tau**2-tau_3**2))-D11)/D0
rho2_mod = A + EARTH_GRAVITATIONAL_PARAMETER*B/(r2_star**3)
rho3_mod = ((6*(D13*tau_3/tau_1 - D23*tau/tau_1)*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*D13*(tau**2-tau_3**2)*tau_3/tau_1)/(6*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*(tau**2-tau_1**2))-D33)/D0
rho1 = Vector(x = rho1_mod*versor_rho1.x, y = rho1_mod*versor_rho1.y, z = rho1_mod*versor_rho1.z)
rho2 = Vector(x = rho2_mod*versor_rho2.x, y = rho2_mod*versor_rho2.y, z = rho2_mod*versor_rho2.z)
rho3 = Vector(x = rho3_mod*versor_rho3.x, y = rho3_mod*versor_rho3.y, z = rho3_mod*versor_rho3.z)

#Cálculo dos r's:
r1 = R1 + rho1
r2 = R2 + rho2
r3 = R3 + rho3

#Coeficientes de Lagrange:
f1 = 1-0.5*EARTH_GRAVITATIONAL_PARAMETER*(tau_1**2)/(r2_star**3)
f3 = 1-0.5*EARTH_GRAVITATIONAL_PARAMETER*(tau_3**2)/(r2_star**3)
g1 = tau_1 - (1/6)*EARTH_GRAVITATIONAL_PARAMETER*(tau_1**3)/(r2_star**3)
g3 = tau_3 - (1/6)*EARTH_GRAVITATIONAL_PARAMETER*(tau_3**3)/(r2_star**3)


#Velocidade:
v2 = (r3*f1 - r1*f3)*((f1*g3-f3*g1)**(-1))


#Resultados:
print('Origens das observações:')
print(f'R1 = {round(R1.x,1)}I + ({round(R1.y,1)})J + ({round(R1.z,1)})K')
print(f'R2 = {round(R2.x,1)}I + ({round(R2.y,1)})J + ({round(R2.z,1)})K')
print(f'R3 = {round(R3.x,1)}I + ({round(R3.y,1)})J + ({round(R3.z,1)})K')
print('-----------------')
print('Versores rhos:')
print(f'versor_rho1 = {round(versor_rho1.x,5)}I + ({round(versor_rho1.y,5)})J + ({round(versor_rho1.z,5)})K')
print(f'versor_rho2 = {round(versor_rho2.x,5)}I + ({round(versor_rho2.y,5)})J + ({round(versor_rho2.z,5)})K')
print(f'versor_rho3 = {round(versor_rho3.x,5)}I + ({round(versor_rho3.y,5)})J + ({round(versor_rho3.z,5)})K')
print('-----------------')
print('Intervalos temporais:')
print(f'tau_1 = {tau_1}')
print(f'tau_3 = {tau_3}')
print(f'tau = {tau}')
print('-----------------')
print('Produtos vetoriais:')
print(f'p1 = {round(p1.x,6)}I + ({round(p1.y,6)})J + ({round(p1.z,6)})K')
print(f'p2 = {round(p2.x,6)}I + ({round(p2.y,6)})J + ({round(p2.z,6)})K')
print(f'p3 = {round(p3.x,6)}I + ({round(p3.y,6)})J + ({round(p3.z,6)})K')
print('-----------------')
print('Constantes (Ds):')
print(f'D0 = {D0}')
print(f'D11 = {D11}     D12 = {D12}     D13 = {D13}')
print(f'D21 = {D21}     D22 = {D22}     D23 = {D23}')
print(f'D31 = {D31}     D32 = {D32}     D33 = {D33}')
print('-----------------')
print('Constantes A e B:')
print(f'A = {A}')
print(f'B = {B}')
print('-----------------')
print('Constantes E e módulo ao quadrado de R2:')
print(f'E = {E}')
print(f'R_2^2 = {R2.magnitude**2}')
print('-----------------')
print('Coeficientes do polinômio (r2^8 + a(r2)^6 + b(r2)^3 + c = 0):')
print(f'a = {round(a/1e7,4)}e+7')
print(f'b = {b}')
print(f'c = {c}')
print('-----------------')
print('valor de r2:')
print(f'r2 = {round(r2_star,1)}')
print('-----------------')
print('Módulos de rhos:')
print(f'rho_1 = {round(rho1_mod,1)}')
print(f'rho_2 = {round(rho2_mod,1)}')
print(f'rho_3 = {round(rho3_mod,1)}')
print('-----------------')
print('Vetores posição no Ref. Inercial:')
print(f'r1 = {round(r1.x,1)}I + ({round(r1.y,1)})J + ({round(r1.z,1)})K')
print(f'r2 = {round(r2.x,1)}I + ({round(r2.y,1)})J + ({round(r2.z,1)})K')
print(f'r3 = {round(r3.x,1)}I + ({round(r3.y,1)})J + ({round(r3.z,1)})K')
print('-----------------')
print('Coeficientes de Lagrange:')
print(f'f1 = {round(f1,5)}      g1 = {round(g1,5)}')
print(f'f3 = {round(f3,5)}      g3 = {round(g3,5)}')
print('-----------------')
print('Velocidade (v2):')
print(f'v2 = {round(v2.x,4)}I + ({round(v2.y,4)})J + ({round(v2.z,4)})K')



Origens das observações:
R1 = 3489.8I + (3430.2)J + (4078.5)K
R2 = 3460.1I + (3460.1)J + (4078.5)K
R3 = 3429.9I + (3490.1)J + (4078.5)K
-----------------
Versores rhos:
versor_rho1 = 0.71643I + (0.68074)J + (-0.1527)K
versor_rho2 = 0.56897I + (0.79531)J + (-0.20917)K
versor_rho3 = 0.4184I + (0.87008)J + (-0.26059)K
-----------------
Intervalos temporais:
tau_1 = -118.1
tau_3 = 119.48000000000002
tau = 237.58
-----------------
Produtos vetoriais:
p1 = -0.025251I + (0.060747)J + (0.162285)K
p2 = -0.044536I + (0.122804)J + (0.338522)K
p3 = -0.020952I + (0.062979)J + (0.182462)K
-----------------
Constantes (Ds):
D0 = -0.0015178807153869954
D11 = 782.1342298696927     D12 = 1646.490450136214     D13 = 887.0871432593711
D21 = 784.7043631501966     D22 = 1651.4927060471437     D23 = 889.596431802398
D31 = 787.2912400554336     D32 = 1656.5251522481512     D33 = 892.1201428189596
-----------------
Constantes A e B:
A = -9.255331042798277
B = 7676363777.219512
-----------------
Constantes E e 

In [5]:
a = Vector(1,1,1)
b = a*2
b.magnitude**2

11.999999999999998